# 3D Numerical integration methods for our problem

In [ ]:
# core stuff
import h5py
import numpy as np
import scipy
from copy import deepcopy

# pytorch
from torch import nn
import torch
# For debugging and development purposes this is now set to float64 ... change for speed on GPUs
torch.set_default_tensor_type(torch.DoubleTensor)

# misc
import sobol_seq
from scipy import integrate
from tqdm import tqdm

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# We generate a low-discrepancy sequence here and keep it in memory (generating it requires some CPU time)
sobol_points_3d = sobol_seq.i4_sobol_generate(3, 1000000)
sobol_points_1d = sobol_seq.i4_sobol_generate(1, 1000000)

## Some test functions to verify functionality

In [ ]:
def f_test(x):
    return 1*x[0]+10*x[1]+100*x[2]

def f1(x):
    return 4 - x**2

def f1_2(x):
    return x**5 - 4*x**4 + 23*x**3 + 4*x**2 - 3*x + 1

def f2(x):
    return 4 - x[:,0]**2 - x[:,1]**2 - x[:,2]**2

def f3(x):
    return 3 * x[:,0] - 5 * x[:,1]**2 + x[:,2]**3

def f4(x):
    return 10 * x[:,0]**3 - 5 * x[:,1]**4 + 3 * x[:,2]**7

In [ ]:
# Naive Montecarlo
def MC(f, dim, N = 1000):
    # We generate randomly points in the [-1,1]^d bounds
    sample_points = torch.rand(N,dim) * 2 - 1
    return 2**dim * torch.sum(f(sample_points)) / N

# Low-discrepancy Montecarlo
def LDMC(f, dim, N = 1000, noise = 1e-5):
    if dim == 1:
        sobol_points = sobol_points_1d
    else:
        sobol_points = sobol_points_3d
    # We generate randomly points in the [-1,1]^3 bounds
    sample_points = torch.tensor(sobol_points[:N,:] * 2 - 1) + torch.rand(N,dim) * noise
    return 2**dim * torch.sum(f(sample_points)) / N

# Utilize Scipy NQUAD
def NQUAD(f, dim, limit = 50,epsabs=100.0,epsrel=100.0):
    if dim==1:
        integration_domain = [[-1,1]]
        func = lambda x: f(torch.tensor([[x]]))
    else:
        integration_domain = [[-1,1],[-1,1],[-1,1]]
        func = lambda x,y,z: f(torch.tensor([[x,y,z]]))
    opts={"limit": limit, "epsabs" : epsabs, "epsrel" : epsrel}
    nquad_out = integrate.nquad(func, integration_domain,opts=opts,full_output=True)
    retval = torch.tensor([[nquad_out[0]]])[0]
    return retval,nquad_out

#1D Trapezoid method in torch
def trapezoid1D(f,N = 2,integration_domain = [[-1,1]],verbose=False):
    #Create grid and assemble evaluation points
    grid_1d = torch.linspace(integration_domain[0][0],integration_domain[0][1],N)
    h = (grid_1d[1] - grid_1d[0])
    eval_points = torch.tensor([x for x in grid_1d])
    
    #Evaluate all points
    evaluations = f(eval_points)
    
    #Compute f0,f1,f2
    f0 = evaluations[0:-1]
    f1 = evaluations[1:]
    areas = h / 2 * (f0 + f1)
    if verbose: print("areas=",areas)
    return torch.sum(areas)

#1D Composite simpson method in torch
def simpson1D(f,N = 3,integration_domain = [[-1,1]],verbose=False):
    if N % 2 != 1: raise(ValueError("N cannot be even due to necessary subdivisions."))
    #Create grid and assemble evaluation points
    grid_1d = torch.linspace(integration_domain[0][0],integration_domain[0][1],N)
    h = (grid_1d[2] - grid_1d[0]) / 2
    eval_points = torch.tensor([x for x in grid_1d])
    if verbose: print("eval_points=",eval_points)
    if verbose: print("h=",h)
    
    #Evaluate all points
    evaluations = f(eval_points)
    if verbose: print("evaluations=",evaluations)
    
    #Compute f0,f1,f2
    f0 = evaluations[0:-2][::2]
    f1 = evaluations[1:-1][::2]
    f2 = evaluations[2:][::2]
    if verbose: print("f0,f1,f2",f0,f1,f2)
    areas = h / 3 * (f0 + 4 * f1 + f2)
    if verbose: print("areas=",areas)
    return torch.sum(areas)

#3D Trapezoid method in torch
def trapezoid3D(f,N = 2,integration_domain = [[-1,1]],verbose=False):
    #Create grid and assemble evaluation points
    grid_1d = torch.linspace(integration_domain[0][0],integration_domain[0][1],N)
    h = (grid_1d[1] - grid_1d[0])
    x,y,z = torch.meshgrid(grid_1d,grid_1d,grid_1d)
    eval_points = torch.stack((x.flatten(),y.flatten(),z.flatten())).transpose(0,1)
    if verbose: print("eval_points=",eval_points)
    if verbose: print("h=",h)
    
    #Evaluate all points
    evaluations = f(eval_points).reshape([N,N,N]) #map to z,y,x
    if verbose: print("evaluations=",evaluations)
    
    # area = h / 2 + (f0 + f2)
    int_x = h / 2 * (evaluations[:,:,0:-1] + evaluations[:,:,1:])
    int_x = torch.sum(int_x,dim=2)
    int_y = h / 2 * (int_x[:,0:-1] + int_x[:,1:])
    int_y = torch.sum(int_y,dim=1)
    int_z = h / 2 * (int_y[0:-1] + int_y[1:])
    int_z = torch.sum(int_z,dim=0)
    if verbose: print("int_x",int_x.shape,int_x)
    if verbose: print("int_y",int_y.shape,int_y)
    if verbose: print("int_z",int_z.shape,int_z)
    return int_z

#1D Composite simpson method in torch
def simpson3D(f,N = 3,integration_domain = [[-1,1]],verbose=False):
    if N % 2 != 1: raise(ValueError("N cannot be even due to necessary subdivisions."))
    #Create grid and assemble evaluation points
    grid_1d = torch.linspace(integration_domain[0][0],integration_domain[0][1],N)
    h = (grid_1d[2] - grid_1d[0]) / 2
    x,y,z = torch.meshgrid(grid_1d,grid_1d,grid_1d)
    eval_points = torch.stack((x.flatten(),y.flatten(),z.flatten())).transpose(0,1)
    if verbose: print("eval_points=",eval_points)
    if verbose: print("h=",h)
    
    #Evaluate all points
    evaluations = f(eval_points).reshape([N,N,N]) #map to z,y,x
    if verbose: print("evaluations=",evaluations.shape,evaluations)
    
    # area = h / 3 + (f0 + 4*f1 + f2)
    int_x = h / 3 * (evaluations[:,:,0:-2][:,:,::2] + 4 * evaluations[:,:,1:-1][:,:,::2] + evaluations[:,:,2:][:,:,::2])
    int_x = torch.sum(int_x,dim=2)
    int_y = h / 3 * (int_x[:,0:-2][:,::2] + 4 * int_x[:,1:-1][:,::2] + int_x[:,2:][:,::2])
    int_y = torch.sum(int_y,dim=1)
    int_z = h / 3 * (int_y[0:-2][::2] + 4 * int_y[1:-1][::2] + int_y[2:][::2])
    int_z = torch.sum(int_z,dim=0)
    if verbose: print("int_x",int_x.shape,int_x)
    if verbose: print("int_y",int_y.shape,int_y)
    if verbose: print("int_z",int_z.shape,int_z)
    return int_z

#Wrapper for different dim
def simpson(f,dim,N,verbose=False):
    if dim==1:
        if N % 2 == 0: 
            N -= 1
        return simpson1D(f=f,N=N,verbose=verbose)
    if dim==3:
        N = int(np.round(np.cbrt(N)))
        if N % 2 == 0: 
            N -= 1
        return simpson3D(f=f,N=N,verbose=verbose)
    else:
        raise(NotImplementedError())

#Wrapper for different dim
def trapezoid(f,dim,N):
    if dim==1:
        return trapezoid1D(f=f,N=N)
    if dim==3:
        N = int(np.round(np.cbrt(N)))
        return trapezoid3D(f=f,N=N)
    else:
        raise(NotImplementedError())

## Test it on the defined functions

In [ ]:
f = f4 #function to test
dim = 3 #dim of the function
ground_truth, _ = NQUAD(f, dim, epsabs = 0.000000000001,epsrel = 0.000000000001) #ground_truth
grid = range(100, 30000, 500) #eval number to test
print(ground_truth)

In [ ]:
mc = []
for g in tqdm(grid):
    mc.append(torch.abs(MC(f, dim, N = g).detach() - ground_truth).numpy())

ld = []
for g in tqdm(grid):
    ld.append(torch.abs(LDMC(f, dim, N = g, noise=1e-5).detach() - ground_truth).numpy())

    
qd,qd_steps = [], []
for limit in tqdm(range(1,50,5)):
    result, info = NQUAD(f,dim,limit=limit)
    qd.append(torch.abs(result.detach() - ground_truth).numpy())
    qd_steps.append(info[2]["neval"])
    
simp = []
for g in tqdm(grid):
    simp.append(torch.abs(simpson(f, dim, N = g).detach() - ground_truth).numpy())
    
trap = []
for g in tqdm(grid):
    trap.append(torch.abs(trapezoid(f, dim, N = g).detach() - ground_truth).numpy())
    
# We plot the results
fig = plt.figure()
plt.semilogy(grid, mc, label = "naive MC")
plt.semilogy(grid, ld, label = "low-discrepancy")
plt.semilogy(qd_steps, qd, label = "nquad")
plt.semilogy(grid, simp, label = "simpsons")
plt.semilogy(grid, trap, label = "trapezoid")
plt.legend()

## Now test on real model

## Load the model

In [ ]:
# We import the data from MPIA containing pseudo-stable asteroid shapes
f = h5py.File('sample_vis_data/sample_01/state_10567.hdf5','r')
f2 = h5py.File('sample_vis_data/sample_01/global.hdf5', 'r')
# The file state_ ... contains the positions of all particles as well as the indices
# of those belonging to a cluster. Here we extract the largest ones.
dims = [(len(f[cluster][()]), cluster) for cluster in f.keys() if 'cluster' in cluster]
largest_clusters = sorted(dims,reverse=True)
# We have ordered the largest asteroids, we now extract positions for one in particular
rank = 4
print("Target: ", largest_clusters[rank][1])
# The particles idxs for this cluster
idx = f[largest_clusters[rank][1]][()]
# The particle radius
radius = f2['radius'][()]
# Particle positions
x_raw = f['x'][()][idx]
y_raw = f['y'][()][idx]
z_raw = f['z'][()][idx]
print("Diameter: ", 2 * radius)
from sklearn.neighbors import NearestNeighbors
# We put xyz in a different shape (point_cloud)
point_cloud = np.append(x_raw, np.append(y_raw,z_raw))
point_cloud = point_cloud.reshape((3,len(x_raw)))
point_cloud = np.transpose(point_cloud)

nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(point_cloud)
distances, indices = nbrs.kneighbors(point_cloud)

print("Minimum distance between particles: ", min(distances[:,1]))
print("Maximum distance between particles: ", max(distances[:,1]))

# We take out particles that are not "touching" at least two neighbours
unstable_points = np.where(distances[:, 3]> 2 * radius * 1.01)[0]
print("Number of unstable points: ", len(unstable_points))
x = np.delete(x_raw, unstable_points, 0)
y = np.delete(y_raw, unstable_points, 0)
z = np.delete(z_raw, unstable_points, 0)
# We subtract the mean so that the origin is the center of figure
x = x - np.mean(x)
y = y - np.mean(y)
z = z - np.mean(z)
# We normalize so that the axes are at most one
max_value = max([max(abs(it)) for it in [x,y,z]])
x = x / max_value
y = y / max_value
z = z / max_value
plot_radius = radius /  max_value  * 3000
# We put xyz in a different shape (point_cloud)
point_cloud = np.append(x, np.append(y,z))
point_cloud = point_cloud.reshape((3,len(x)))
point_cloud = np.transpose(point_cloud)
point_cloud = torch.tensor(point_cloud)
# This will create the labels for the supervised learning
def U_L(target_points, point_cloud):
    retval=torch.empty(len(target_points),1)
    for i, target_point in enumerate(target_points):
        retval[i] = torch.mean(1./torch.norm(torch.sub(point_cloud,target_point), dim=1))
    return - retval 

# All encodings work taking as input a tensor (N, 3) containing the cartesian coordinates of N points
# and returning a tensor of (N, M) that can be used as input to the ANN

# Encoding N.1 (directional encoding):
# x = [x,y,z] is encoded as [ix, iy, iz, r]
def directional_encoding(sampled_points):
    unit = sampled_points / torch.norm(sampled_points,dim=1).view(-1,1)
    return torch.cat((unit, torch.norm(sampled_points,dim=1).view(-1,1)), dim=1)

# Encoding N.2 (positional encoding):
# x = [x,y,z] is encoded as [sin(pi x), sin(pi y), sin(pi z), cos(pi x), cos(pi y), cos(pi z), sin(2 pi x), ....]
def positional_encoding(sampled_points, N = 4):
    retval = torch.cat((torch.sin(np.pi * dummy[:,0]).view(-1,1), torch.cos(np.pi * dummy[:,0]).view(-1,1), torch.sin(np.pi * dummy[:,1]).view(-1,1), torch.cos(np.pi * dummy[:,1]).view(-1,1), torch.sin(np.pi * dummy[:,2]).view(-1,1), torch.cos(np.pi * dummy[:,2]).view(-1,1)), dim=1)
    for i in range(1, N):
        retval = torch.cat((retval, torch.sin(2**i * np.pi * dummy[:,0]).view(-1,1), torch.cos(2**i * np.pi * dummy[:,0]).view(-1,1), torch.sin(2**i * np.pi * dummy[:,1]).view(-1,1), torch.cos(2**i * np.pi * dummy[:,1]).view(-1,1), torch.sin(2**i * np.pi * dummy[:,2]).view(-1,1), torch.cos(2**i * np.pi * dummy[:,2]).view(-1,1)), dim=1)
    return retval

# Encoding N.3 (direct encoding):
def direct_encoding(sampled_points):
    return sampled_points
        
# Encoding N.4 (spherical coordinates). These can be used with positional encoding to create effectively harmonics
def spherical_coordinates(sampled_points):
    phi = torch.atan2(dummy[:,1], dummy[:,0]) / np.pi
    r = torch.norm(dummy, dim=1)
    theta = torch.div(dummy[:,2], r)
    return torch.cat((r.view(-1,1), phi.view(-1,1), theta.view(-1,1)), dim=1)
# Encoding choosen
encoding = directional_encoding

# Network initialization scheme (note that if xavier uniform is used all outputs will start at, roughly 0.5)
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.uniform_(m.bias.data, -0.0, 0.0)

# Network architecture. Note that the dimensionality of the first linear layer must match the output
# of the encoding chosen
n_neurons = 100
model = nn.Sequential(
          nn.Linear(4,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,n_neurons),
          nn.ReLU(),
          nn.Linear(n_neurons,1),
          nn.Sigmoid(),
        )

# Applying our weight initialization
_  = model.apply(weights_init)

# IF YOU NOW WANT TO LOAD THE ALREADY TRAINED NETWORK UNCOMMENT HERE.
## It is important that the network architecture is compatible, otherwise this will fail
model.load_state_dict(torch.load("models/" + largest_clusters[rank][1]))

## Setup integration methods

In [ ]:
# We generate a low-discrepancy sequence here and keep it in memory (generating it requires some CPU time)
sobol_points = sobol_seq.i4_sobol_generate(3, 500000)

In [ ]:
# Naive Montecarlo
def U_Pmc(target_points, model, N = 3000):
    # We generate randomly points in the [-1,1]^3 bounds
    sample_points = torch.rand(N,3) * 2 - 1
    nn_inputs = encoding(sample_points)
    rho = model(nn_inputs)
    retval=torch.empty(len(target_points),1)
    # Only for the points inside we accumulate the integrand (MC method)
    for i, target_point in enumerate(target_points):
        retval[i] = torch.sum(rho/torch.norm(target_point - sample_points, dim=1).view(-1,1)) / N
    return  - 8 * retval

# Low-discrepancy Montecarlo
def U_Pld(target_points, model, N = 3000, noise = 1e-5):
    # We generate randomly points in the [-1,1]^3 bounds
    sample_points = torch.tensor(sobol_points[:N,:] * 2 - 1) + torch.rand(N,3) * noise
    nn_inputs = encoding(sample_points)
    rho = model(nn_inputs)
    retval=torch.empty(len(target_points),1)
    # Only for the points inside we accumulate the integrand (MC method)
    for i, target_point in enumerate(target_points):
        retval[i] = torch.sum(rho/torch.norm(target_point - sample_points, dim=1).view(-1,1)) / N
    return  - 8 * retval

def f(sample_points,target_point,model):
    nn_inputs = encoding(sample_points)
    nn_inputs[nn_inputs!=nn_inputs] = 0.0 #set Nans to 0
    rho = model(nn_inputs)
    value = rho/torch.norm(target_point - sample_points, dim=1).view(-1,1)
    return value.detach()

def U_quadrature(target_points, model, integration_domain = [[-1,1],[-1,1],[-1,1]], limit = 2,epsabs=100.0,epsrel=100.0):
    opts={"limit": limit, "epsabs" : epsabs, "epsrel" : epsrel}
    retval = torch.empty(len(target_points),1)
    nquad_out = []
    for i, target_point in enumerate(target_points):
        func = lambda x,y,z: f(torch.tensor([[x,y,z]]),target_point,model)
        nquad_out.append(integrate.nquad(func, integration_domain,opts=opts,full_output=True))
        retval[i] = -nquad_out[-1][0]
    return retval,nquad_out
    
def U_simp(target_points, model, N,verbose=False):
    retval = torch.empty(len(target_points),1)
    for i, target_point in enumerate(target_points):
        func = lambda x: f(x,target_point,model)
        retval[i] = simpson(func,dim=3,N=N,verbose=verbose)
    return -retval

def U_trap(target_points, model, N,verbose=False):
    if N % 2 == 0: 
        N -= 1
    retval = torch.empty(len(target_points),1)
    for i, target_point in enumerate(target_points):
        func = lambda x: f(x,target_point,model)
        retval[i] = trapezoid(func,dim=3,N=N)
    return -retval

def U_trap_opt(target_points, model, N,verbose=False):
    N = int(np.round(np.cbrt(N))) #approximate subdivisions
    retval = torch.empty(len(target_points),1) #init result vector
    
    #Create grid and assemble evaluation points
    grid_1d = torch.linspace(-1,1,N)
    h = (grid_1d[1] - grid_1d[0])
    x,y,z = torch.meshgrid(grid_1d,grid_1d,grid_1d)
    eval_points = torch.stack((x.flatten(),y.flatten(),z.flatten())).transpose(0,1)
    if verbose: print("eval_points=",eval_points)
    if verbose: print("h=",h)
    
    #Evaluate Rho on the grid
    nn_inputs = encoding(eval_points) #Encode grid
    nn_inputs[nn_inputs!=nn_inputs] = 0.0 #set Nans to 0
    rho = model(nn_inputs)
    
    for i, target_point in enumerate(target_points):
        
        f_values = rho/torch.norm(target_point - eval_points, dim=1).view(-1,1).detach()

        #Evaluate all points
        evaluations = f_values.reshape([N,N,N]) #map to z,y,x
        if verbose: print("evaluations=",evaluations)

        # area = h / 2 + (f0 + f2)
        int_x = h / 2 * (evaluations[:,:,0:-1] + evaluations[:,:,1:])
        int_x = torch.sum(int_x,dim=2)
        int_y = h / 2 * (int_x[:,0:-1] + int_x[:,1:])
        int_y = torch.sum(int_y,dim=1)
        int_z = h / 2 * (int_y[0:-1] + int_y[1:])
        int_z = torch.sum(int_z,dim=0)
        if verbose: print("int_x",int_x.shape,int_x)
        if verbose: print("int_y",int_y.shape,int_y)
        if verbose: print("int_z",int_z.shape,int_z)
    
        retval[i] = int_z
    return -retval.detach()

In [ ]:
U_trap(target_points, model, N=1000)

In [ ]:
U_trap_opt(target_points, model, N=1000)

## Create test points

In [ ]:
# Here we create some target points where to compute the potential
torch.manual_seed(42)
N_try = 10
target_points = (torch.rand(N_try,3)*2-1)*1.1
a = torch.logical_and((target_points[:,0]>-1),(target_points[:,0]<1))
b = torch.logical_and((target_points[:,1]>-1),(target_points[:,1]<1))
c = torch.logical_and((target_points[:,2]>-1),(target_points[:,2]<1))
d = torch.logical_and(torch.logical_or(a,b), c)
target_points=target_points[d]
print("Target point is: ", target_points)

In [ ]:
#For seed 42 
# ground_truth = torch.tensor([[-0.6497132186723324],[-1.0661419291444170],[-0.5982705907123077],[-1.0310878380126529],[-0.9525779194515801],[-0.8621937637431156],[-0.8976110804434343],[-1.1594308542434955],[-0.8590986260083082]])
# ground_truth, _ = U_quadrature(target_points, model, epsabs = 0.000001,epsrel = 0.000001)
ground_truth = U_trap_opt(target_points, model, N=1000000)

torch.set_printoptions(precision=16)
print("Ground truth is: ", ground_truth)
torch.set_printoptions(precision=4)

## Profile runtime performance

In [ ]:
N = 10000

In [ ]:
%timeit U_Pmc(target_points, model, N = N)

In [ ]:
%timeit U_Pld(target_points, model, N = N)

In [ ]:
%timeit U_trap(target_points, model, N = N)

In [ ]:
%timeit U_trap_opt(target_points, model, N = N)

## Profile accuracy vs evals

In [ ]:
grid = range(1000, 500000, 25000)
mc,ld,simp,trap = [],[],[],[]
for g in tqdm(grid):
    mc.append(torch.abs(U_Pmc(target_points, model, N = g).detach() - ground_truth).numpy())
    ld.append(torch.abs(U_Pld(target_points, model, N = g, noise=1e-5).detach() - ground_truth).numpy())
#     simp.append(torch.abs(U_simp(target_points, model, N = g).detach() - ground_truth).numpy())
    trap.append(torch.abs(U_trap_opt(target_points, model, N = g).detach() - ground_truth).numpy())
if target_points.shape[0] > 1:
    mc = np.mean(mc,axis=1)
    ld = np.mean(ld,axis=1)
#     simp = np.mean(simp,axis=1)
    trap = np.mean(trap,axis=1)    

In [ ]:
# We plot the results
fig = plt.figure()
plt.semilogy(grid, mc, label = "naive MC")
plt.semilogy(grid, ld, label = "low-discrepancy")
plt.semilogy(grid, trap, label = "trapezoid quadrature")
# plt.semilogy(grid, simp, label = "composite s-impson quadrature")
plt.legend()

In [ ]:
# Here we create some target points where to compute the potential
torch.manual_seed(42)
N_try = 1
target_points = (torch.rand(N_try,3)*2-1)*1.1
a = torch.logical_and((target_points[:,0]>-1),(target_points[:,0]<1))
b = torch.logical_and((target_points[:,1]>-1),(target_points[:,1]<1))
c = torch.logical_and((target_points[:,2]>-1),(target_points[:,2]<1))
d = torch.logical_and(torch.logical_or(a,b), c)
target_points=target_points[d]
print("Target point is: ", target_points)

In [ ]:
# This is slow, run only to generate again the plot below
grid = range(1000, 250000, 25000)
mc_val,ld_val,simp_val,trap_val = [],[],[],[]
for g in tqdm(grid):
#     mc_val.append(torch.abs(U_Pmc(target_points, model, N = g).detach()).numpy().squeeze())
    ld_val.append(torch.abs(U_Pld(target_points, model, N = g, noise=1e-5).detach()).numpy().squeeze())
    simp_val.append(torch.abs(U_simp(target_points, model, N = g).detach()).numpy().squeeze())
    trap_val.append(torch.abs(U_trap_opt(target_points, model, N = g).detach()).numpy().squeeze())

In [ ]:
# We plot the results
fig = plt.figure()
# plt.semilogy(grid, mc_val, label = "naive MC")
plt.semilogy(grid, ld_val, label = "low-discrepancy")
# plt.semilogy(qd_steps, qd, label = "nquad")
plt.semilogy(grid, trap_val, label = "trapezoid quadrature")
plt.semilogy(grid, simp_val, label = "composite simpson quadrature")
plt.legend()

In [ ]:
# We plot the results
fig = plt.figure()
# plt.semilogy(grid, mc_val, label = "naive MC")
plt.semilogy(grid, np.abs(ld_val-ld_val[-1]), label = "low-discrepancy")
# plt.semilogy(qd_steps, qd, label = "nquad")
plt.semilogy(grid, np.abs(trap_val-trap_val[-1]), label = "trapezoid quadrature")
plt.semilogy(grid, np.abs(simp_val-simp_val[-1]), label = "composite simpson quadrature")
plt.legend()